# Question 2:


In [63]:
# !pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip3 install pandas
# !pip3 install numpy
# !pip3 install sklearn
# !pip3 install tqdm
# !pip3 install transformers
# !pip install newspaper3k

In [64]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from torch import nn
from torch.optim import Adam
from transformers import GPT2Model, GPT2Tokenizer
from tqdm import tqdm
import pandas as pd
import sqlite3
import newspaper
import random
import csv
import langdetect
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# 1.

## Scraping technological and political news using newspaper3k package and assigning label to them as technology and political.

In [65]:
conn = sqlite3.connect('news_articles.db')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS articles (id INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT, text TEXT, label TEXT)')

tech_websites = ['https://www.techradar.com/', 'https://www.theverge.com/', 'https://www.wired.com/',
                 'https://www.tomshardware.com/']

political_websites = ['https://www.politico.com/',
                      'https://www.nytimes.com/section/politics']

for website in tech_websites + political_websites:
    print(f'Scraping {website}...')
    paper = newspaper.build(website)
    count = 0

    for article in paper.articles:
        if count == 200:
            break
        try:
            article.download()
            article.parse()
            title = article.title
            text = article.text
            label = 'technology' if website in tech_websites else 'political'
            c.execute("INSERT INTO articles (title, text, label) VALUES (?, ?, ?)", (title, text, label))
            count += 1
        except Exception as e:
            print(f"Error processing article: {str(e)}")

conn.commit()
c.execute("SELECT title, text, label FROM articles WHERE label='technology' OR label='political'")
results = c.fetchall()

random.shuffle(results)
with open('news_dataset.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['text', 'label'])
    for result in results:
        writer.writerow([result[1], result[2]])

conn.close()

Scraping https://www.techradar.com/...


CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.techradar.com/feeds
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.techradar.com/feed


Scraping https://www.theverge.com/...


CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.theverge.com/feeds
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.theverge.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.theverge.com/rss


Error processing article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.fortnite.com/news/race-without-limits-with-rocket-racing-in-fortnite on URL https://www.fortnite.com/news/race-without-limits-with-rocket-racing-in-fortnite
Error processing article: Article `download()` failed with ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')) on URL https://www.thegamer.com/neil-newbon-wins-best-performance-at-the-game-awards-2023/
Error processing article: Article `download()` failed with 403 Client Error: Forbidden for url: https://genius.com/Nobuo-uematsu-one-winged-angel-final-fantasy-vii-lyrics on URL https://genius.com/Nobuo-uematsu-one-winged-angel-final-fantasy-vii-lyrics
Scraping https://www.wired.com/...


...

CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.tomshardware.com/rss
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.tomshardware.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.tomshardware.com/feeds


Scraping https://www.politico.com/...


CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.politico.com/%7B%7Bpermalink%7D%7D
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.politico.com/%7B%7BflagURL%7D%7D
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.politico.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.politico.com/feeds


Scraping https://www.nytimes.com/section/politics...


CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.nytimes.com/feeds
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://www.nytimes.com/feed


In [66]:
news_data = pd.read_csv('news_dataset.csv')

In [69]:
news_data.shape
# this data has 724 articles, so we need to filter out the political and technology news using our keywords

(724, 2)

In [68]:
# drop missing values
news_data.dropna(inplace=True)

In [70]:
# Defining the keywords for each category to filter them
technology_keywords = ['AI', 'GPT-4', 'GPT', 'Transformers', 'Gemini', 'OpenAI']
political_keywords = ['Obama', 'Trump', 'Biden', 'Joe Biden', 'Republican', 'Democracy', 'president']

technology_articles = news_data[news_data['label'] == 'technology']
technology_articles = technology_articles[technology_articles['text'].str.contains('|'.join(technology_keywords), case=False)]

political_articles = news_data[news_data['label'] == 'political']
political_articles = political_articles[political_articles['text'].str.contains('|'.join(political_keywords), case=False)]

# Sample 312 articles from each technology
technology_sample = technology_articles.sample(312, random_state=42)
result_df = pd.concat([technology_sample, political_articles])
result_df = result_df.sample(frac=1, random_state=42).reset_index(drop=True)
result_df.to_csv('news_data.csv', index=False)

In [71]:
result_df.label.value_counts()

technology    312
political     140
Name: label, dtype: int64

Each article has around 1000+ words. There are 452 articles in total with two categories-technology and political(312 and 140 articles each). Let's check if the articles are in english or not for ease in pre-processing.

In [80]:
df = pd.read_csv('/content/news_data.csv')

In [81]:
def detect_language(text):
    try:
        return langdetect.detect(text)
    except:
        return 'unknown'
df['language'] = df['text'].apply(detect_language)

In [82]:
df.language.value_counts()

en    448
ja      4
Name: language, dtype: int64

In [83]:
df.label.value_counts()

technology    312
political     140
Name: label, dtype: int64

In [84]:
df = df[df['language'] == 'en']
df = df.drop(columns=['language'])

In [85]:
df.head()

,text,label
0,Beeper says “fix coming soon” for its Android ...,technology
1,You could say that Fortnite is entering a new ...,technology
2,"According to the IOC, eight Russian and three ...",political
3,That partnership will be put to the test this ...,political
4,"Transcript\n\n[Pilot] Attention, passengers.\n...",technology


In [86]:
df.label.value_counts()

technology    308
political     140
Name: label, dtype: int64

There are 448 articles in total, and technology -308,
political - 140 articles.

## Preprocessing data

We  build a custom Dataset class to tokenize our news dataset and store them into containers for batch training.For GPT-2 we need to do padding to the left, because we need to use the last token for prediction.

In [88]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
labels = {
    "technology": 0,
    "political": 1
         }

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text,
                                padding='max_length',
                                max_length=128,
                                truncation=True,
                                return_tensors="pt") for text in df['text']]
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

# 2.

## Split training-test dataset

In [95]:
np.random.seed(42)

df_train, df_test = np.split(df.sample(frac=1, random_state=42), [int(0.8*len(df))])
df_train, df_val = np.split(df_train.sample(frac=1, random_state=42), [int(0.8*len(df_train))])

print(len(df_train), len(df_val), len(df_test))

286 72 90


We have 448 articles in total and we split the training and test by 80:20 and again split the training into train and val(80:20). So we have 286 articles for train , 72 for validation and 90 for test.

# 3.

# Fine tuning pre-trained model: GPT-2

Adding a linear layer on top of GPT-2's 12 layers of decoders with its output dimension equals our number of categories(2 in this case).

In [96]:
class SimpleGPT2SequenceClassifier(nn.Module):
    def __init__(self, hidden_size: int, num_classes:int ,max_seq_len:int, gpt_model_name:str):
        super(SimpleGPT2SequenceClassifier,self).__init__()
        self.gpt2model = GPT2Model.from_pretrained(gpt_model_name)
        self.fc1 = nn.Linear(hidden_size*max_seq_len, num_classes)
    def forward(self, input_id, mask):
        gpt_out, _ = self.gpt2model(input_ids=input_id, attention_mask=mask, return_dict=False)
        batch_size = gpt_out.shape[0]
        linear_output = self.fc1(gpt_out.view(batch_size,-1))
        return linear_output

In [97]:
def train(model, train_data, val_data, learning_rate, epochs):
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input["input_ids"].squeeze(1).to(device)

            model.zero_grad()

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1)==train_label).sum().item()
            total_acc_train += acc

            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1)==val_label).sum().item()
                total_acc_val += acc

            print(
            f"Epochs: {epoch_num + 1} | Train Loss: {total_loss_train/len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}")

EPOCHS = 1
model = SimpleGPT2SequenceClassifier(hidden_size=768, num_classes=2, max_seq_len=128, gpt_model_name="gpt2")
LR = 1e-5

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 143/143 [00:12<00:00, 11.84it/s]


Epochs: 1 | Train Loss:  0.472             | Train Accuracy:  0.671             | Val Loss:  0.164             | Val Accuracy:  0.861


We train it for 1 epoch because GPT-2 is already a large model trained on lots of parameters and we acheive a good validation accuracy(0.861) by training on 1 epoch itself.

Evaluate on test data

In [98]:
def evaluate(model, test_data):
    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    predictions_labels = []
    true_labels = []
    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
            true_labels += test_label.cpu().numpy().flatten().tolist()
            predictions_labels += output.argmax(dim=1).cpu().numpy().flatten().tolist()
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return true_labels, predictions_labels
true_labels, pred_labels = evaluate(model, df_test)

Test Accuracy:  0.867


In [100]:
# save trained model
torch.save(model.state_dict(), "./gpt2-text-classifier-model.pt")

In [101]:
# load trained model
model_new = SimpleGPT2SequenceClassifier(hidden_size=768, num_classes=2, max_seq_len=128, gpt_model_name="gpt2")
model_new.load_state_dict(torch.load("./gpt2-text-classifier-model.pt"))
model_new.eval()

SimpleGPT2SequenceClassifier(
  (gpt2model): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (fc1): Linear(in_features=98304, out_features=2, bias=True)
)

# Predicting on new data from a different website to evaluate on the fine tuned model.

In [102]:
example = """
U.S. chip giant Nvidia (NVDA.O) will discuss cooperation deals on semiconductors with Vietnamese tech companies and authorities in a meeting on Monday in Hanoi, an invitation letter to participants seen by Reuters showed.
The southeast Asian country, which is home to large chip assembling factories including Intel's (INTC.O) biggest globally, is trying to expand into chip designing and possibly chip-making as trade tensions between the United States and China create opportunities for Vietnam in the strategic industry.
"""
lower = " ".join(example.lower().split())
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
model_inp = tokenizer(lower, padding='max_length', max_length=128, truncation=True, return_tensors="pt")

In [103]:
mask = model_inp['attention_mask'].cpu()
input_id = model_inp["input_ids"].squeeze(1).cpu()
output = model_new(input_id, mask)

In [105]:
prob = torch.nn.functional.softmax(output, dim=1)[0]
prob

tensor([0.9295, 0.0705], grad_fn=<SelectBackward0>)

In [106]:
labels_map = {
    0: "technology",
    1: "political",
         }
pred = labels_map[output.argmax(dim=1).item()]
pred

'technology'

Test Accuracy is 86.7 percent. We can improve this by scraping more news articles from different websites and increase the train size of the model. Additionally different pre-trained transformer models (including BERT, BART, RoBERTa) or a transformer model fine-tuned for news articles (in HuggingFace )can be used to see which one performs better on large training data and this use case.

Another way would be to use different context of technological data and political data rather than just the news articles to make the model truly understand such aspects.

# References:



1.  Dataset created - https://drive.google.com/drive/folders/1SOezYpvKoHJj7dw_v8b_G6ZnEr-jZ5P3?usp=sharing
2. https://huggingface.co/gpt2
3.   https://discuss.huggingface.co/t/fine-tuning-gpt2-for-movie-script-generation-in-pytorch/23906/3
4. https://github.com/haocai1992/GPT2-News-Classifier?tab=readme-ov-file


